# Set up env

In [1]:
import carla
import numpy as np
from src.simulator import Simulator
from src.agent import NCPAgent
from src.model import Model, Trainer

import torchvision
from IPython.display import display
import sys
import os
import torch

sys.path.append("CARLA_SIM/PythonAPI/carla/")
from agents.navigation.basic_agent import BasicAgent


# Training process

In [2]:
from src.model import Model, Trainer

In [1]:
import carla
from random import random

random_rotation = 3 * random() - 1.5
mapping = {
    'Town01_opt': {
        'loc': carla.Location(x=176.589493, y=133.239151, z=0.300000), # 15 spawn point
        'rotation': carla.Rotation(0, 0 + random_rotation, 0)
    },
    'Town02_opt': {
        'loc': carla.Location(x=193.779999, y=142.190002, z=0.500000), # 15 spawn point
        'rotation': carla.Rotation(0, -90 + random_rotation, 0)
    },
    'Town03_opt': {
        'loc': carla.Location(x=-88.710991, y=-119.565231, z=0.275307), # 1 spawn point
        'rotation': carla.Rotation(0, 90 + random_rotation, 0)
    },
    'Town04_opt': {
        'loc': carla.Location(405.320374, -48.450779, 0.281942), #222 spawn point
        'rotation': carla.Rotation(0, -90 + random_rotation, 0)
    },
}
print(random_rotation)

0.8856674893382861


In [4]:
simulator = Simulator(world_name='Town04_opt', dump_data=True, debug=True)

In [5]:
import time

simulator.world.get_spectator()#.set_transform()
    # carla.Transform(
    #     location=carla.Location(x=398.7934265136719,
    #                             y=-56.03200912475586,
    #                             z=3.37939715385437)))

simulator.spawn_car_with_camera(
    rel_coordinates=carla.Location(x=1.2, z=1.9), # camera coords
    vehicle_coordinates=mapping[simulator.world_name]['loc'],
    vehicle_rotation=mapping[simulator.world_name]['rotation']
)
vehicle = simulator.get_vehicle()



output_size = 2
units = 19
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

ncp = Model(output_size, units)
ncp.to(device)
if not os.path.isdir(f'out/{simulator.world_name}'):
    os.mkdir(f'out/{simulator.world_name}')
with open(f'out/{simulator.world_name}/data.txt', 'a+') as f:
    f.write(f'timestamp start = {time.time()}\n')
agent = NCPAgent(simulator, ncp, target_speed=10)

next_waypoint  = [simulator.world.get_map().get_waypoint(vehicle.get_location(),
                                                    project_to_road=True,
                                                    lane_type=(carla.LaneType.Driving))]

waypoints = []
dist_between_waypoints = 15
waypoint_num = 50
# waypoint_num = 350
# waypoint_num = 100
# waypoint_num = 10
for _ in range(waypoint_num):
    waypoints.append(next_waypoint[-1])
    # simulator.world.get_spectator().set_transform(next_waypoint[-1].transform)
    next_waypoint = next_waypoint[-1].next(dist_between_waypoints)

dest_idx = 2
dest = waypoints[dest_idx].transform.location
agent.set_destination(dest)
agent.set_target_speed(10)
agent.ignore_traffic_lights(active=True)
agent.ignore_stop_signs(active=True)

cuda
alloc!
Camera stream started


In [6]:
# # os.rmdir('./out/')

idx = 1
tmp = 0
waypoint = waypoints[idx]
to_PIL = torchvision.transforms.ToPILImage()

# loss_func = torch.nn.functional.mse_loss
# optimizer = torch.optim.Adam(ncp.parameters(), lr=0.001)

# # basic_agent = BasicAgent(vehicle=vehicle)
# trainer = Trainer(ncp, loss_func, optimizer)

while True:
    tmp += 1
    # if vehicle.get_location().distance(dest) < 0.5:
    #     print(f"Destination reached")
    #     break
    if (vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints / 4) or \
        (vehicle.get_location().distance(waypoint.transform.location) >= dist_between_waypoints * 2 and \
         vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints * 3):
        print(f'Waypoint {idx} was reached')
        if idx + 1 >= len(waypoints):
            print("The target has been reached, stopping the simulation")
            break
        waypoint = waypoints[idx + 1]
        idx += 1

    # control, movement, raw_data, out_tensor = agent.run_step()
    control, _, raw_data= agent.run_step(dump_data=True)
    # if raw_data is not None:
    #     trainer.train(raw_data, torch.tensor(control.steer))
    #     print(out_tensor)
    # if tmp % 150000:
    #     print(f"control = {control}")
    vehicle.apply_control(control)
    if agent.simulator.image_frame is not None:
        with open(f'out/{simulator.world_name}/data.txt', 'a+') as f:
            f.write(f'{agent.simulator.image_frame} : {control.steer}\n')

    
    if agent.done():
        if dest_idx < waypoint_num - 1:
            dest_idx += 10
            dest_idx = min(dest_idx, waypoint_num - 1)
            print(f'Intermediate destination reached. Moving to waypoint {dest_idx}')
            agent.is_done = False
            agent.set_destination(waypoints[dest_idx].transform.location)
            continue

        print("The target has been reached, stopping the simulation")
        break
vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))

Waypoint 1 was reached
Waypoint 2 was reached
Intermediate destination reached. Moving to waypoint 12
Waypoint 3 was reached
Waypoint 4 was reached
Waypoint 5 was reached
Waypoint 6 was reached
Waypoint 7 was reached
Waypoint 8 was reached
Waypoint 9 was reached
Waypoint 10 was reached
Waypoint 11 was reached
Intermediate destination reached. Moving to waypoint 22
Waypoint 12 was reached
Waypoint 13 was reached
Waypoint 14 was reached
Waypoint 15 was reached
Waypoint 16 was reached
Waypoint 17 was reached
Waypoint 18 was reached
Waypoint 19 was reached
Waypoint 20 was reached
Waypoint 21 was reached
Intermediate destination reached. Moving to waypoint 32
Waypoint 22 was reached
Waypoint 23 was reached
Waypoint 24 was reached
Waypoint 25 was reached
Waypoint 26 was reached
Waypoint 27 was reached
Waypoint 28 was reached
Waypoint 29 was reached
Waypoint 30 was reached
Waypoint 31 was reached
Intermediate destination reached. Moving to waypoint 42
Waypoint 32 was reached
Waypoint 33 was r

In [7]:
simulator.destroy_all()

Camera stream stopped


In [18]:
len(waypoints)
len(set(waypoints))

600

# training

In [1]:
from src.model import Model, Trainer

import torch

output_size = 4
units = 19
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model = Model(output_size, units)
model.to(device)

loss_func = torch.nn.functional.mse_loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # + l2 reg 1e-5?

data_path = "out"
annotations_file = f"{data_path}/concat_data.txt"
trainer = Trainer(model,
                  loss_func,
                  optimizer,
                  annotations_file=annotations_file,
                  img_dir=data_path,
                  test_size=0.5,
                  random_state=42,
                  stb_weights=[1, 0, 0])
trainer.train(epochs=10, batch_size=64)


cuda
alloc!
Epoch 0


Train: 100%|█████████████████████████████████████████████████████████████████████████████████▊| 493/494 [02:58<00:00,  2.78it/s]/mnt/hard_drive/habkaffee_part/programming/masters_paper/venv/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
Test: 100%|███████████████████████████████████████████████████████████████████████████████████| 494/494 [01:31<00:00,  5.42it/s]


LOSS train 0.0009016382391564548 valid 7.276442920556292e-05
Epoch 1


Test: 100%|███████████████████████████████████████████████████████████████████████████████████| 494/494 [01:33<00:00,  5.29it/s]


LOSS train 0.00560436537489295 valid 3.855139220831916e-05
Epoch 2


...

LOSS train 0.0008648432558402419 valid 1.6861906260601245e-05
Epoch 8


Test: 100%|███████████████████████████████████████████████████████████████████████████████████| 494/494 [01:45<00:00,  4.68it/s]


LOSS train 0.00018378239474259317 valid 1.648719080549199e-05
Epoch 9


Test: 100%|███████████████████████████████████████████████████████████████████████████████████| 494/494 [01:38<00:00,  5.00it/s]


LOSS train 0.0008742011850699782 valid 2.2520727725350298e-05


In [2]:
model.save_model(f'model/all_towns_10.pth')

In [ ]:
# ncp.save_model(f'./model/pretrained_wp{waypoint_num}.pth')

# test

In [2]:
import carla
import numpy as np
from src.simulator import Simulator
from src.agent import NCPAgent
from src.model import Model, Trainer

from random import uniform

import torchvision
from IPython.display import display
import sys
import os
import torch
sys.path.append("CARLA_SIM/PythonAPI/carla/")
from agents.navigation.basic_agent import BasicAgent


In [3]:
simulator = Simulator(world_name='Town04_opt', debug=False, dump_data=False)

In [4]:
simulator.spawn_car_with_camera(
    rel_coordinates=carla.Location(x=1.2, z=1.9), # camera coords
    vehicle_coordinates=mapping[simulator.world_name]['loc'],
    vehicle_rotation=mapping[simulator.world_name]['rotation']
)
vehicle = simulator.get_vehicle()
print(f'Transforms : {vehicle.get_transform()}')

output_size = 4
units = 19
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

Transforms : Transform(Location(x=405.320374, y=-48.450779, z=0.230191), Rotation(pitch=0.000000, yaw=-89.114311, roll=0.000000))
cuda


In [5]:
checkpoint_name = "model/all_towns_10.pth"

In [6]:
ncp = Model(output_size, units)
ncp.load_model(checkpoint_name)
ncp = ncp.to(device=device)
ncp.eval()
agent = NCPAgent(simulator, ncp, target_speed=10)

alloc!
Camera stream started


In [7]:
# ncp = Model(output_size, units)
# ncp = ncp.to(device=device)
# agent = NCPAgent(simulator, ncp, target_speed=10)

next_waypoint  = [simulator.world.get_map().get_waypoint(vehicle.get_location(),
                                                    project_to_road=True,
                                                    lane_type=(carla.LaneType.Driving))]

waypoints = []
dist_between_waypoints = 20
waypoint_num = 5
for _ in range(waypoint_num):
    waypoints.append(next_waypoint[-1])
    next_waypoint = next_waypoint[-1].next(dist_between_waypoints)

dest_idx = 2
dest = waypoints[dest_idx].transform.location
agent.set_destination(dest)
agent.ignore_traffic_lights(True)
agent.ignore_stop_signs(True)
agent.set_target_speed(10)

idx = 0
while True:
    control, out, raw_data = agent.run_step()
    print(control.steer, out[0][:2], out[0][1]+out[0][0])
    # break
    new_control = carla.VehicleControl(steer=control.steer, throttle=control.throttle, brake=control.brake)
    vehicle.apply_control(new_control)

    if simulator.dump_data and (agent.simulator.image_frame is not None):
        with open(f'out/{simulator.world_name}/data.txt', 'a+') as f:
            f.write(f'{agent.simulator.image_frame} : {control.steer}\n')

    if agent.done():
        if dest_idx < waypoint_num - 1:
            dest_idx += 10
            dest_idx = min(dest_idx, waypoint_num - 1)
            print(f'Intermediate destination reached. Moving to waypoint {dest_idx}')
            agent.is_done = False
            agent.set_destination(waypoints[dest_idx].transform.location)
            continue

        print("The target has been reached, stopping the simulation")
        break
    idx += 1
vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))
simulator.destroy_all()

-0.014976978302001953 tensor([-0.0061,  0.0165], device='cuda:0', grad_fn=<SliceBackward0>) tensor(0.0104, device='cuda:0', grad_fn=<AddBackward0>)
-0.015015381388366222 tensor([-0.0170,  0.0130], device='cuda:0', grad_fn=<SliceBackward0>) tensor(-0.0041, device='cuda:0', grad_fn=<AddBackward0>)
-0.01503458246588707 tensor([-0.0173,  0.0121], device='cuda:0', grad_fn=<SliceBackward0>) tensor(-0.0052, device='cuda:0', grad_fn=<AddBackward0>)
-0.015053783543407917 tensor([-0.0173,  0.0119], device='cuda:0', grad_fn=<SliceBackward0>) tensor(-0.0053, device='cuda:0', grad_fn=<AddBackward0>)
-0.015072984620928764 tensor([-0.0182,  0.0118], device='cuda:0', grad_fn=<SliceBackward0>) tensor(-0.0064, device='cuda:0', grad_fn=<AddBackward0>)
-0.015092185698449612 tensor([-0.0185,  0.0118], device='cuda:0', grad_fn=<SliceBackward0>) tensor(-0.0067, device='cuda:0', grad_fn=<AddBackward0>)
-0.015111387707293034 tensor([-0.0185,  0.0117], device='cuda:0', grad_fn=<SliceBackward0>) tensor(-0.0068, 

In [6]:
simulator.destroy_all()

Camera stream stopped


- l1 loss instead of mse
- sum l1 + l2?
- Training framework (pt lightning, catalyst)